In [ ]:
!pip install librosa

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
from google.colab import files

print("Libraries imported. Please upload your audio file now.")
uploaded = files.upload()

# Get the filename of the uploaded file
if uploaded:
    for fn in uploaded.keys():
        uploaded_filename = fn
        print(f"User uploaded file '{uploaded_filename}'")
else:
    print("No file uploaded.")

In [ ]:
SR = 22050 # Define a standard sampling rate

# 1. Load the uploaded audio file
audio_time_series, sr = librosa.load(uploaded_filename, sr=None)
print(f"Original sampling rate: {sr} Hz")

# 2. Resample the audio if the sampling rate is not SR
if sr != SR:
    audio_time_series = librosa.resample(y=audio_time_series, orig_sr=sr, target_sr=SR)
    sr = SR
    print(f"Resampled to: {sr} Hz")

# 3. Compute the Short-Time Fourier Transform (STFT)
N_FFT = 2048 # Number of FFT points
HOP_LENGTH = 512 # Number of samples between successive frames
stft_result = librosa.stft(audio_time_series, n_fft=N_FFT, hop_length=HOP_LENGTH)

# 4. Convert the amplitude spectrogram to decibels (dB)
db_spectrogram = librosa.amplitude_to_db(np.abs(stft_result), ref=np.max)

# 5. Create a temporary filename for the spectrogram image
temp_spectrogram_filename = "temp_spectrogram.png"

# 6. Generate a Mel spectrogram
plt.figure(figsize=(10, 4))
librosa.display.specshow(db_spectrogram, sr=sr, hop_length=HOP_LENGTH, y_axis='mel', x_axis='time')

# 7. Add a color bar
plt.colorbar(format='%+2.0f dB', label='dB')
plt.title('Mel-frequency spectrogram')
plt.tight_layout()

# 8. Save the generated spectrogram plot
plt.savefig(temp_spectrogram_filename, bbox_inches='tight', pad_inches=0)
print(f"Spectrogram saved as '{temp_spectrogram_filename}'")

# 9. Close the plot to free up memory
plt.close()


In [ ]:
import os
import matplotlib.pyplot as plt

# Install and import YOLO (Ultralytics)
!pip install -q ultralytics
from ultralytics import YOLO

# Path to the trained model on Google Drive
trained_model_drive_path = '/content/drive/MyDrive/fft/trained_model.pt'

# Step 1: Load the model from Google Drive
if os.path.exists(trained_model_drive_path):
    print(f"Loading trained model from '{trained_model_drive_path}'...")
    model = YOLO(trained_model_drive_path)
    print("Trained model loaded from Google Drive for inference.")

    # Step 2: Run inference on the generated spectrogram
    # temp_spectrogram_filename should be defined from previous steps
    if 'temp_spectrogram_filename' in locals() and os.path.exists(temp_spectrogram_filename):
        print(f"Running inference on '{temp_spectrogram_filename}'...")
        results = model(temp_spectrogram_filename)
        print("Inference completed.")

        # Step 3: Display detection results
        for r in results:
            r.show()  # Display the results directly
    else:
        print("Error: Temporary spectrogram file not found or "
              "variable 'temp_spectrogram_filename' is not defined. "
              "Please ensure previous steps ran successfully.")
else:
    print(f"Error: Trained model not found at '{trained_model_drive_path}'. "
          "Please check the path or make sure the model is saved correctly.")
